# **Proyecto final**

# <font color=8C64AA> **1st WBDS LA Camp "Introduction to Bioinformatics and Data Science" 2023**

Cathcarth, Marilina

## **OBJETIVO**

El objetivo que me planteo en este proyecto es automatizar el análisis de los datos provenientes de simulaciones de Teoría Molecular, haciendo la comparación de los resultados de cada simulación con los datos experimentales utilizando métricas de precisión.  

## **RELEVANCIA**

Mi tesis doctoral, titulada *“Diseño racional de modificaciones superficiales en nanopartículas para controlar su adsorción de proteínas y estabilidad coloidal”*, tiene como objetivo general el desarrollo de herramientas teórico-computacionales, que serán contrastadas dinámicamente con datos experimentales, para permitir dilucidar la relación entre las propiedades superficiales de nanopartículas y su estabilidad coloidal y la adsorción de proteínas sobre estas en medios complejos.

Uno de los objetivos específicos del plan de trabajo es el desarrollo de una teoría molecular (TM) para estudiar las propiedades superficiales y la estabilidad coloidal de nanopartículas de sílica (SNPs) con diferentes funcionalizaciones en soluciones diluidas.

La TM es un método de Mecánica Estadística y Termodinámica basado aproximaciones de campo medio autoconsistentes dentro de la teoría del funcional de la densidad (clásica). Este enfoque permite el estudio sistemático en función de las condiciones del medio de sistemas macromoleculares y nanoparticulados abarcando escalas espaciales del orden de cientos de nanómetros. La información molecular se incorpora al método mediante modelos de grano grueso que permiten considerar el tamaño, la forma, la distribución de carga, las configuraciones y estados químicos de las diferentes especies químicas. La formulación del potencial termodinámico incluye todas las contribuciones fisicoquímicas relevantes. El modelo de TM es implementado en un código Fortran paralelizado usando OpenMPI.

### **Parametrización de una superficie de sílica que regula carga**

En la figura se ilustra un esquema del sistema, una superficie plana (z = 0) en contacto con una solución que contiene agua, iones de hidróxido e hidronio, cationes de sodio y aniones de cloruro. Lejos de la superficie (z → ∞), se controla experimentalmente la concentración de cada una de estas especies; las variables independientes de nuestros cálculos son el pH y la concentración de NaCl, Csalt. La superficie se modifica grupos hidroxilo (OH) que surgen de los silanoles terminales, se considera la posibilidad de que existan dos tipos diferentes de OH (para fines prácticos aquí los llamaremos A y B). Tanto las especies libres pequeñas como los OH superficiales son modelados como esferas (Cathcarth et al., 2022).

![Esquema](https://drive.google.com/uc?id=1NxM7XdYEDEMV0Ds5e2Hgh6fVI122xwmq)


El objetivo es obtener los parámetros del modelo que mejor ajusten los resultados teóricos con resultados experimentales. Se cuenta con datos experimentales de densidad de carga superficial (SCD) en función del pH para cuatro concentraciones de NaCl: 22, 55, 109 y 216 mM (Goel, 2018). Los parámetros que se desean obtener son: relación entre A y B (%A), constante de disociación ácida de los OH (pKaA y pKaB), energía de interacción del agua con la especie neutra OH (wA y wB) y por último la constante del equilibrio de la reacción del Na+ con la especie desprotonada O- (pKNa), esta última consideramos que es igual para ambos (A y B). Tanto el binding del agua como el equilibrio del sodio son opcionales, y hay simulaciones que no cuentan con alguno de esos parámetros.

Cada simulación se realiza con una combinación diferente entre ellos (1764 simulaciones) y esto se hace para cada concentración de sal (7056 simulaciones). Para generar los directorios, los inputs, correr las simulaciones y hacer el postprocesado de datos se cuenta con scripts de bash y fortran que automatizan el proceso. La idea de este proyecto es automatizar el siguiente paso, el análisis de las curvas.

Al final de las corridas se genera un directorio llamado “**compare**” que contiene un directorio para cada concentración de sal (**022**, **055**, **109** y **216**), dentro de los cuales hay archivos CSV las curvas de SCD vs pH para cada simulación y la curva experimental. Los archivos con los datos experimentales están nombrados con 4 dígitos (**0022.csv**, **0055.csv**, **0109.csv** y **0216.csv**), por otro lado, los archivos de las simulaciones están nombrados con los parámetros separados por “**_**” (**%A_pKaA_pKaB_wA_wB_pKNa.csv**) en caso de simulaciones que no tienen alguno de los parámetros (pKaA, wA, wB o pKNa) se informa con una x en el lugar correspondiente.



## **PASOS PARA SOLUCIONAR EL PROBLEMA**

1. Importar los paquetes necesarios (os, pandas, sklearn.metrics, math, shutil y matplotlib.pyplot). - L9-L14

2. Pedirle al usuario que ingrese la ruta del directorio "**compare**" (en caso de que no esté definida en el script). - L16-L38

3. Crear el directorio de análisis ("**analysis**"). - L40-51

4. Generar la ruta para cada directorio dentro de **compare** (para cada concentración de sal). - L55-L64

5. Hacer un loop para ingresar a cada directorio y leer los archivos *CSV* (pH vs SCD) en su interior. - L66-L69

6. Generar un DataFrame para cada directorio que contenga la información de todos los archivos leídos. - L74-L93

  - Hacer un loop por cada archivo y:
    - Agregar las dos columnas del archivo experimental.
    -	Agregar la segunda columna de cada archivo teórico
    - Renombrar cada columna teórica agregando el número de simulación

   **DataFrame**: 'sim_df'
   
     - Columnas: pH, SCD_exp, sim_0, sim_1, ..., sim_1763

7. Generar un DataFrame para cada directorio que contenga la información de los nombres de cada archivo de simulación (los parámetros de cada simulación). - L95-L132

  - Identificar los archivos de simulación (los que tienen 6 parámetros separados por '_')
  - Hacer un loop por cada uno de esos archivos
  - Obtener cada parámetro del nombre y agregarlo a una lista para cada parámetro
  - Generar un diccionario con esas 6 listas que mapee el nombre del parámetro con la lista correspondiente
  - Generar el DataFrame a partir de ese diccionario
   
  **DataFrame**: 'params_df'
	  
    - Columnas: %A, pK-A, pK-B, w-A, w-B, pKNa
    - Filas: sim_0 a sim_1763

8. Comparar cada curva teórica con la experimental mediante el uso de métricas de regresión. - L137-L166

 -  Generar una lista con las métricas a utilizar
 -  Para cada directorio de concentración, generar un DataFrame con una métrica por columna
 -  Calcular las métricas para cada columna de simulación del DataFrame 'sim_df' (la columna experimental es la 2 'SCD')
 -  Generar una nueva primer columna para identificar cada simulación 'sim'
   
  **DataFrame**: 'df_metrics'
 - Columnas: sim, MAE, MSE, RMSE, R2
 - Filas: sim_0 a sim_1763

9. Crear archivos *CSV* a partir de los 3 DataFrames en el directorio '**analysis**', incluir en el comienzo del nombre la concentración de sal. - L168-L212 

(comentario: el archivo de los parámetros podría no guardarse, son iguales para todas las concentraciones de sal)

 - Guardar los 3 DataFrames en archivos *CSV* en el directorio de la concentración de sal correspondiente
 - Renombrar los DataFrames para incluir el nombre del directorio de concentración 
 - Mover los DataFrames renombrados al directorio '**analysis**'

10. Generar un DataFrame para cada métrica. - L219-L269

 - Leer los 4 DataFrames de las métricas y almacenarlos en un diccionario (la llave es la concentración de sal)
 - Para cada métrica crear un DataFrame que contenga columnas con los resultados para cada concentración de sal
 - Agregar una sexta columna con la suma de las métricas
 - Ordenar el DataFrame en función a la nueva columna (en orden descendente si la métrica es R2 y ascendente para las otras 3)
 - Guardar el DataFrame en un archivo *CSV* con el nombre de la métrica

 **DataFrame**: (MAE.csv, MSE.csv, RMSE.csv y R2.csv)
  - Columnas: sim, 022, 055, 109, 216
	- Filas: sim_0 a sim_1763

10. Dar al usuario la opción de ver el análisis de los resultados. - L274-L449

 - Permitir que el usuario elija la métrica que quiere utilizar
 - Leer el archivo de la métrica elegida
 - Mostrar los resultados de la mejor simulación (la de índice 0) para cada concentración de sal
 - Obtener el id de la mejor simulación para mostrar los parámetros correspondientes
 - Graficar, si el usuario quiere, las curvas de la mejor simulación y las curvas experimentales para cada concentración de sal
 - Guardar el gráfico en un archivo *PNG*
 - Dar la opción al usuario de ver los resultados de otra métrica

FIN	


## **CONCLUSIÓN:**

### **Resultados**:

La simulación que mejores resultados obtiene según la métrica MAE es: sim_1300

  Los resultados para cada concentración de sal son:

      22 mm: 0,0055313725490196
      55 mm: 0,0072726923076923
     109 mm: 0,0099886538461538
     216 mm: 0,011838448275862

  Los parámetros de simulación son:

       % A: 20
     pK-A: 6,5
     pK-B: 8,5
      w-A: 0
      w-B: 2.5
     pKNa: +1,86

---
La simulación que mejores resultados obtiene según la métrica MSE es: sim_1300

  Los resultados para cada concentración de sal son:

      22 mm: 3.650239999999999e-05
      55 mm: 6.171733076923078e-05
     109 mm: 0,0001613722211538
     216 mm: 0,0001806315293103

  Los parámetros de simulación son:

       % A: 20
     pK-A: 6,5
     pK-B: 8,5
      w-A: 0
      w-B: 2.5
     pKNa: +1,86

---
La simulación que mejores resultados obtiene según la métrica RMSE es: sim_1300

  Los resultados para cada concentración de sal son:

      22 mm: 0,0060417216089455
      55 mm: 0,0078560378543659
     109 mm: 0,0127032366408662
     216 mm: 0,0134399229651938

  Los parámetros de simulación son:

       % A: 20
     pK-A: 6,5
     pK-B: 8,5
      w-A: 0
      w-B: 2.5
     pKNa: +1,86

---
La simulación que mejores resultados obtiene según la métrica R2 es: sim_1300

  Los resultados para cada concentración de sal son:

      22 mm: 0,9883272375522558
      55 mm: 0,9815804245120692
     109 mm: 0,9590094837982588
     216 mm: 0,9653690856365624

  Los parámetros de simulación son:

       % A: 20
     pK-A: 6,5
     pK-B: 8,5
      w-A: 0
      w-B: 2.5
     pKNa: +1,86

---
Con las 4 métricas obtengo que la mejor simulación es la 1300

La mejor combinación de parámetros probados para describir la superficie de sílica es:

       % A: 20
     pK-A: 6,5
     pK-B: 8,5
      w-A: 0
      w-B: 2.5
     pKNa: +1,86

![gráfico](https://drive.google.com/uc?id=1da3dfrNYobRUwx813frVWGqZ2KZqUoZH)


Comentarios: Las curvas experimentales deberían ir a cero a bajos pHs, proximamente buscaré otros datos experimentales para hacer el análisis.
